In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import statistics as st 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.compose import make_column_selector
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost.sklearn import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from math import sqrt
import seaborn as sns
import category_encoders as ce

In [24]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train = train[train['price'] != 0]

In [25]:
#Change all to uppercase
train['property_type']=train['property_type'].str.upper()

#Change all to uppercase
test['property_type']=test['property_type'].str.upper()

In [26]:
property_to_keep = test['property_type'].unique()
train.property_type.isin(property_to_keep)

train[~train.property_type.isin(property_to_keep)]

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
635,432676,3 bed house for sale in the glencaird residences,dalvey road,the glencaird residences,LAND ONLY,freehold,2000.0,3.0,3.0,107,...,unspecified,"3, 4, 5, 6 br",12.0,https://www.99.co/singapore/houses/the-glencai...,1.315477,103.819725,0,nassim,tanglin,101640000.0
1575,178157,5 bed house for sale in fuyong estate,jalan uji,fuyong estate,LAND ONLY,99-year leasehold,NaN,5.0,5.0,5726,...,unfurnished,"3, 4, 5, 6 br",NaN,https://www.99.co/singapore/houses/fuyong-esta...,1.358099,103.768404,0,nature reserve,bukit panjang,1257900.0
19090,727605,2 bed house for sale in 265 outram road,outram road,265 outram road,SHOPHOUSE,freehold,2019.0,2.0,2.0,657,...,unspecified,"1, 2 br",17.0,https://www.99.co/singapore/condos-apartments/...,1.283937,103.835488,0,pearl's hill,outram,1621100.0


In [27]:
categories_not_in_test = ['LAND ONLY', 'SHOPHOUSE']
train = train[train['property_type'].isin(categories_not_in_test) == False] 

In [29]:
train['property_type'].unique()

array(['HDB 4 ROOMS', 'HDB', 'CONDO', 'BUNGALOW', 'TERRACED HOUSE',
       'HDB EXECUTIVE', 'APARTMENT', 'SEMI-DETACHED HOUSE', 'HDB 3 ROOMS',
       'EXECUTIVE CONDO', 'CORNER TERRACE', 'HDB 5 ROOMS', 'LANDED',
       'HDB 2 ROOMS', 'CLUSTER HOUSE', 'TOWNHOUSE', 'CONSERVATION HOUSE',
       'WALK-UP', 'GOOD CLASS BUNGALOW'], dtype=object)

In [30]:
train['property_type'] = train['property_type'].replace(
    ['CLUSTER HOUSE', 'CONSERVATION HOUSE', 'LANDED', 'TOWNHOUSE', 'GOOD CLASS BUNGALOW'],
    'RARELANDED'
)

test['property_type'] = test['property_type'].replace(
    ['CLUSTER HOUSE', 'CONSERVATION HOUSE', 'LANDED', 'TOWNHOUSE', 'GOOD CLASS BUNGALOW'],
    'RARELANDED'
)
train['property_type'] = train['property_type'].replace(
    ['CORNER TERRACE', 'SEMI-DETACHED HOUSE'],
    'SEMID'
)

test['property_type'] = test['property_type'].replace(
    ['CORNER TERRACE', 'SEMI-DETACHED HOUSE'],
    'SEMID'
)
train['property_type'] = train['property_type'].replace(
    ['HDB 3 ROOMS', 'HDB 4 ROOMS', 'HDB 5 ROOMS', 'HDB 2 ROOMS'],
    'HDB'
)

test['property_type'] = test['property_type'].replace(
    ['HDB 3 ROOMS', 'HDB 4 ROOMS', 'HDB 5 ROOMS', 'HDB 2 ROOMS'],
    'HDB'
)
train['property_type'] = train['property_type'].replace(
    ['WALK-UP', 'APARTMENT'],
    'CONDO'
)

test['property_type'] = test['property_type'].replace(
    ['WALK-UP', 'APARTMENT'],
    'CONDO'
)

In [ ]:
Rlanded = df[df['property_type'] == 'RARELANDED']
condo = df[df['property_type'] == 'CONDO']
hdb = df[df['property_type'] == 'HDB']
Econdo = df[df['property_type'] == 'EXECUTIVE CONDO']
Ehdb = df[df['property_type'] == 'HDB EXECUTIVE']